In [1]:
import os
# import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from setuptools import setup
from setuptools.command.install import install
from IPython.display import Markdown as md
from Bio import SeqIO
# from scipy import stats
import csv
from itertools import product

# Get barcodes

In [2]:
data_fp ='/ru-auth/local/home/abrenner/myscratch/helping_labmates/'

In [3]:
df = pd.read_csv(data_fp+'umi_demultiplex.csv')

In [4]:
barcode_combos_pre = (list(product(df['start_barcode'], df['end_barcode'])))

In [5]:
barcode_combos = [barcode_combos_pre[i] + ('combo'+str(i)+'.fastq',) for i in range(0,len(barcode_combos_pre))]

In [6]:
len(barcode_combos)

784

# Read in FASTQ and distibute

In [ ]:
# fp = '/lustre/fs4/home/amansisido/scratch/RICC-Y/SSRP-Seq-2/fastq_19bp/Attempt1_BLC2fastq/Subsampled/'
# counter = 0
# for seq_record in SeqIO.parse(fp+'Undetermined_S0_I1_001.fastqtest_reads.fastq', """fastq"""):
#     desc_string = seq_record.description[-28:]
#     for combo in barcode_combos:
# #         outfile = open(data_fp + combo[2],'w')
#         criteria1 = combo[0][:8] == desc_string[:8]
#         criteria2= combo[1]== desc_string[-8:]
#         if criteria1 & criteria2:
# #             print(seq_record.index)
#             SeqIO.write(seq_record, data_fp + combo[2], "fastq")
#             counter = counter + 1
# #             outfile.close()
# print(counter)

In [ ]:
# fp = '/lustre/fs4/home/amansisido/scratch/RICC-Y/SSRP-Seq-2/fastq_19bp/Attempt1_BLC2fastq/Subsampled/'
# counter = 0
# for seq_record in SeqIO.parse(fp+'Undetermined_S0_I1_001.fastqtest_reads.fastq', """fastq"""):
#     desc_string = seq_record.description[-28:]
#     for combo in barcode_combos:
# #         outfile = open(data_fp + combo[2],'w')
#         criteria1 = combo[0][:8] == desc_string[:8]
#         criteria2= combo[1]== desc_string[-8:]
#         if criteria1 & criteria2:
# #             print(seq_record.index)
#             with open(data_fp + combo[2], "w") as output_handle:
#                 SeqIO.write(seq_record, output_handle, "fastq")
# #             counter = counter + 1


In [ ]:
# fp = '/lustre/fs4/home/amansisido/scratch/RICC-Y/SSRP-Seq-2/fastq_19bp/Attempt1_BLC2fastq/Subsampled/'
# counter = 0
# for seq_record in SeqIO.parse(fp+'Undetermined_S0_I1_001.fastqtest_reads.fastq', """fastq"""):
#     desc_string = seq_record.description[-28:]
#     for combo in barcode_combos:
# #         outfile = open(data_fp + combo[2],'w')
#         criteria1 = combo[0][:8] == desc_string[:8]
#         criteria2= combo[1]== desc_string[-8:]
#         if criteria1 & criteria2:
# #             print(seq_record.index)
#             if error
#             sequences = [i for i in SeqIO.parse(data_fp + combo[2], """fastq""")]
#             sequences.append(seq_record)
#             SeqIO.write(sequences, data_fp + combo[2], "fastq")
#             counter = counter + 1
# #             outfile.close()
# print(counter)

In [7]:
barcode_combos

[('TTACCGACNNNNNNNNNNN', 'CGTATTCG', 'combo0.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'TCAAGGAC', 'combo1.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'AAGCACTG', 'combo2.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GCAATGGA', 'combo3.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'CAATCGAC', 'combo4.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GGCGTTAT', 'combo5.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GTTAAGGC', 'combo6.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'CCTATACC', 'combo7.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'CTCCTAGA', 'combo8.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GTTACGCA', 'combo9.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'CTAGCAAG', 'combo10.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'ATCTCGCT', 'combo11.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GTGCCATA', 'combo12.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'GGTGATTC', 'combo13.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'CACCTTAC', 'combo14.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'TTCTCTCG', 'combo15.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'TAGTTGCG', 'combo16.fastq'),
 ('TTACCGACNNNNNNNNNNN', 'AGTCTGTG', 'combo17.fastq'),
 ('TTACCGACNNNNNNNNN

Make sure fastqs are empty

In [10]:
fp = '/lustre/fs4/home/amansisido/scratch/RICC-Y/SSRP-Seq-2/fastq_19bp/Attempt1_BLC2fastq/Subsampled/'
counter = 0

# Delete existing fastqs to ensure you do not write into existing files
for combo in barcode_combos:
#     print(data_fp + combo[1])
    if not os.path.isfile(data_fp + combo[2]):
        continue
    os.remove(data_fp + combo[2]) 

for seq_record in SeqIO.parse(fp+'Undetermined_S0_I1_001.fastqtest_reads.fastq', """fastq"""):
    counter = counter + 1
    desc_string = seq_record.description[-28:]
    for combo in barcode_combos:
#         outfile = open(data_fp + combo[2],'w')
        criteria1 = combo[0][:8] == desc_string[:8]
        criteria2= combo[1]== desc_string[-8:]
        if criteria1 & criteria2:
#           # If the file does not exist yet, just save the seq_record to a new file with that name
            if not os.path.isfile(data_fp + combo[2]):
                SeqIO.write(seq_record, data_fp + combo[2], "fastq")
            # If the files does exist, read it in and add onto it
            else:
                sequences = [i for i in SeqIO.parse(data_fp + combo[2], """fastq""")]
                sequences.append(seq_record)
                SeqIO.write(sequences, data_fp + combo[2], "fastq")
#             outfile.close()
print(counter)

1000


In [ ]:
# sequences = [i for i in SeqIO.parse(fp+'Undetermined_S0_I1_001.fastqtest_reads.fastq', """fastq""")]

In [ ]:
# len(sequences)

In [ ]:
# type(sequences[0])

In [ ]:
# sequences.append(seq_record)

In [ ]:
# len(sequences)

In [ ]:
# type(seq_record)

In [ ]:
# seq_record

In [ ]:
counter - 124*len(barcode_combos_pre)

In [ ]:
[i for i in SeqIO.parse(data_fp + combo[2], """fastq""")]